# Feed-forward сети

Итак, давайте потренируемся тренировать нейронные сети прямого распространения (так как делали на паре)
При этом попробуем создать свою функцию активации на одном из слоев

In [ ]:
# Сделаем необходимые импорты

In [1]:
import torch
import numpy as np

from torch import nn
import torchvision

import torch.nn.functional as F

import matplotlib.pyplot as plt

from torch import optim

import torchvision.transforms as transforms

from tqdm import tqdm

In [ ]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10

In [4]:
train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)


image, label = train_dataset[0]
print (image.size())
print (label)

Files already downloaded and verified
torch.Size([3, 32, 32])
19


In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [6]:
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False)

Files already downloaded and verified


In [ ]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации

In [21]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 2 * hidden_dim)
        self.fc2 = nn.Linear(2 * hidden_dim, 4 * hidden_dim)
        self.fc3 = nn.Linear(4 * hidden_dim, 8 * hidden_dim)
        self.fc4 = nn.Linear(8 * hidden_dim, 4 * hidden_dim)
        self.fc5 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.fc6 = nn.Linear(2 * hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc2(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc3(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc4(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc5(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc6(x)
        #x = max(0.1*x, 0.5*x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc2(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc3(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc4(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc5(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc6(x)
        x = F.softmax(x)
        return x

In [22]:
net = Net(3072, 100, 100)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# Запустить обучение (по аналогии с тем, что делали на паре)

In [24]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.002
[1,   301] loss: 0.691
[1,   601] loss: 0.690


 10%|█         | 1/10 [00:16<02:28, 16.45s/it]

[2,     1] loss: 0.002
[2,   301] loss: 0.631
[2,   601] loss: 0.611


 20%|██        | 2/10 [00:33<02:12, 16.58s/it]

...

 80%|████████  | 8/10 [02:12<00:32, 16.47s/it]

[9,     1] loss: 0.002
[9,   301] loss: 0.496
[9,   601] loss: 0.495


 90%|█████████ | 9/10 [02:28<00:16, 16.43s/it]

[10,     1] loss: 0.002
[10,   301] loss: 0.486
[10,   601] loss: 0.487


100%|██████████| 10/10 [02:45<00:00, 16.51s/it]
